- 다양한 곱셈 시나리오를 통해, right sm todense latency 와 sm * dm latency 측정
- 모든 측정 결과들에 대해, [ right sm todense latency / sm * dm latency ] 계산
- 전체 곱셈 시나리오 중, 20%의 시나리오에서 sm todense latency가 실제 sm * dm latency에 비해  큰 비율을 차지하는 것을 확인
    - 해당 결과는 matrix size, density 등의 다양한 원인들이 존재하기 때문에, 정확한 원인을 설명하긴 어려움